In [1]:
!pip install einops torchvision

In [2]:
!pip install torch
!pip install huggingface-hub
!pip install pillow
!pip install torchvision
!pip install sentencepiece
!pip install bitsandbytes
!pip install fairscale fire blobfile einops
!pip install git+https://github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/accelerate
























































































































  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-zzodzfvh

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-zzodzfvh



  Resolved https://github.com/huggingface/transformers to commit 6432ad8bb5dec9c7ece1041767c9e208ff6b4cbb

  Installing build dependencies ... done

  Getting requirements to build wheel ... done

  Preparing metadata (pyproject.toml) ... done



















  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10035552 sha256=0d3cc914b4251337dae56f4eac1eff1ca62df5ae832eef9f554ac6710eef1cbb

  Stored in directory: /tmp/pip-ephem-wheel-cache-yqbbxtia/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b

Successfully built transformers


  Attempting uninstall: transformers

    Found existing installation:

In [3]:
from transformers import AutoModelForCausalLM, AutoProcessor
from PIL import Image
import torch

# Carica il processore
processor = AutoProcessor.from_pretrained(
    'allenai/Molmo-7B-D-0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

# Carica il modello
model = AutoModelForCausalLM.from_pretrained(
    'allenai/Molmo-7B-D-0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
import os
import torch
import time
import re
from transformers import GenerationConfig
from PIL import Image

input_dir = "PPE-detection-no-ppe-v2/test/images"
output_dir = "output/Molmo-7B-D-0924/test_set_predictions_results"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)
file_list = [filename for filename in os.listdir(input_dir) if filename.endswith(".jpg") or filename.endswith(".png")]
num_files = len(file_list)
limit = num_files  # Calcola un terzo del numero di file

# Loop through each image in the directory
for filename in file_list:
    file_path = os.path.join(input_dir, filename)

    # Open image
    image = Image.open(file_path)
    image = image.resize((448, 448))
    #print(f'Sto elaborando: {filename}')
    inputs = processor.process(
        images=[image],
        text=   
            "Are there PPEs in the scene? "
            "Are there blue, orange or yellow high visibility vests in the scene? "
            "Are there gloves? "
            "Are there security boots? "
            "Are there hard hats? "
            "Count people who doesn't wear PPE in the picture, I'll make you an example:\n"
            "    - People: {number of people}\n"
            "    - no-Hard_hats: {number of Hard_hats}\n"
            "    - no-Gloves: {number of Gloves}\n"
            "    - no-Reflective_vests: {number of Reflective_vests}\n"
            "    - no-Security_boots: {number of Security_boots}\n"
    )
    # Sposta gli input sul dispositivo del modello
    inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}

    start_time = time.time()
    output = model.generate_from_batch(
        inputs,
        GenerationConfig(max_new_tokens=2000, stop_strings="<|endoftext|>"),
        tokenizer=processor.tokenizer
    )
    elapsed_time = time.time() - start_time
    #print(f'Elapsed time: {elapsed_time}\n')

    generated_tokens = output[0, inputs['input_ids'].size(1):]
    generated_text = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)
    #print(f"Descrizione dell'immagine: {generated_text}\n")

    output_file_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_output.txt")
    with open(output_file_path, "w") as output_file:
        output_file.write(f"Image filename: {filename}\n{generated_text}\n")
        output_file.write(f"Processing time: {elapsed_time:.4f} seconds\n")

***To reduce the quantity of memory utilizing bfloat16***

In [5]:
'''model.to(dtype=torch.bfloat16)
inputs["images"] = inputs["images"].to(torch.bfloat16)
noutput = model.generate_from_batch(inputs,
   GenerationConfig(max_new_tokens=200, 
stop_strings="<|endoftext|>"),  
tokenizer=processor.tokenizer)'''

'model.to(dtype=torch.bfloat16)\ninputs["images"] = inputs["images"].to(torch.bfloat16)\nnoutput = model.generate_from_batch(inputs,\n   GenerationConfig(max_new_tokens=200, \nstop_strings="<|endoftext|>"),  \ntokenizer=processor.tokenizer)'

In [5]:
mapping = [
    ("Worker", "People:"),
    ("no-vest", "no-Reflective_vests:"),
    ("no-shoes", "no-Security_boots:"),
    ("no-gloves", "no-Gloves:"),
    ("no-helmet", "no-Hard_hats:"),
]

def calculate_metrics(ground_truth, predictions, mapping):
    metrics = {"TP": 0, "FP": 0, "FN": 0}
    for class_name, model_key in mapping:
        if model_key is None:
            pred_count = 0  # No prediction for this class
        else:
            pred_count = predictions.get(model_key, 0)
        true_count = ground_truth.get(class_name, 0)
        
        if pred_count == true_count:
            metrics["TP"] += true_count
        elif true_count > pred_count:
            metrics["TP"] += pred_count
            metrics["FN"] += (true_count - pred_count)
        else:
            metrics["TP"] += true_count
            metrics["FP"] += (pred_count - true_count)
    
    return metrics

def calculate_metrics_per_DPI(ground_truth, predictions, mapping, dpi):
    metrics = {"TP": 0, "FP": 0, "FN": 0}
    for class_name, model_key in mapping:
        if model_key is None or dpi != model_key.strip(":"):
            continue  # Skip if the current class does not match the DPI we're focusing on
        
        pred_count = predictions.get(model_key, 0)
        true_count = ground_truth.get(class_name, 0)
        
        if pred_count == true_count:
            metrics["TP"] += true_count
        elif true_count > pred_count:
            metrics["TP"] += pred_count
            metrics["FN"] += (true_count - pred_count)
        else:
            metrics["TP"] += true_count
            metrics["FP"] += (pred_count - true_count)
    
    return metrics

    

def read_ground_truth(file_path):
    ground_truth = {}
    with open(file_path, 'r') as f:
        for line in f:
            elements = line.split()
            class_id = int(elements[0])
            if class_id == 2:
                ground_truth["no-gloves"] = ground_truth.get("no-gloves", 0) + 1
            elif class_id == 3:
                ground_truth["no-helmet"] = ground_truth.get("no-helmet", 0) + 1
            elif class_id == 6:
                ground_truth["Worker"] = ground_truth.get("Worker", 0) + 1
            elif class_id == 4:
                ground_truth["no-shoes"] = ground_truth.get("no-shoes", 0) + 1
            elif class_id == 5:
                ground_truth["no-vest"] = ground_truth.get("no-vest", 0) + 1
    return ground_truth


def extract_json_from_file(file_path):
    result = {}
    found = False
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.lstrip()
            if line.startswith('-'):
                DPI = ''
                found = True
                words = line.split()
                flag = True
                for word in words:
                    if not flag: 
                        break
                    if word == 'People:' or word == 'no-Gloves:' or word == 'no-Security_boots:' or word == 'no-Reflective_vests:' or word == 'no-Hard_hats:':
                        DPI = word
                        found = True
                    if word.isdigit():
                        result[DPI] = result.get(DPI, 0) + int(word)
                        flag = False
                    if word == 'Yes':
                        result[DPI] = result.get(DPI, 0) + 1
                        flag = False
                    if word == 'No' or word.lower() == 'not':
                        result[DPI] = result.get(DPI, 0)
                        flag = False
        if not found:
            result['People:'] = result.get('People:', 0)
            result['no-Gloves:'] = result.get('no-Gloves:', 0)
            result['no-Security_boots:'] = result.get('no-Security_boots:', 0)
            result['no-Reflective_vests:'] = result.get('no-Reflective_vests:', 0)
            result['no-Hard_hats:'] = result.get('no-Hard_hats:', 0)
    #print(f'result: {result}')
    return result
            
                

def filter_greater_than_zero(data):
    return {key: value for key, value in data.items() if value > 0}

def calculate_classification_metrics(metrics):
    TP = metrics["TP"]
    FP = metrics["FP"]
    FN = metrics["FN"]
    
    accuracy = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
    }

def save_metrics_to_file(metrics, output_folder, file_name):
    output_path = os.path.join(output_folder, f"{file_name}_metrics.json")
    with open(output_path, 'w') as f:
        json.dump(metrics, f, indent=4)
    #print(f"Metrics saved to {output_path}")
    
def calculate_final_metrics(all_metrics):
    total_accuracy = 0
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    n = len(all_metrics)
    # Somma i valori delle metriche per ciascun file
    for metric in all_metrics:
        total_accuracy += metric['accuracy']
        total_precision += metric['precision']
        total_recall += metric['recall']
        total_f1_score += metric['f1_score']

        
    # Calcola la media per ciascuna metrica
    final_metrics = {
        "final_accuracy": total_accuracy / n, 
        "final_precision": total_precision / n, 
        "final_recall": total_recall / n, 
        "final_f1_score": total_f1_score / n
    }
    return final_metrics

In [6]:
import json
import os

ground_truth_folder = "PPE-detection-no-ppe-v2/test/labels"
predictions_folder = "output/Molmo-7B-D-0924/test_set_predictions_results"    
output_folder = "output/Molmo-7B-D-0924/metrics_output"
output_folder_final = "output/Molmo-7B-D-0924/metrics_output_final"
os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder_final, exist_ok=True)
DPI = ['People', 'no-Hard_hats', 'no-Gloves', 'no-Reflective_vests', 'no-Security_boots']

# Calcolo del tempo medio di elaborazione
total_processing_time = 0
file_count = 0

gt_files = sorted(os.listdir(ground_truth_folder))
pred_files = sorted(os.listdir(predictions_folder)) 
all_metrics = []
for gt_file, pred_file in zip(gt_files, pred_files):
    base_filename = os.path.splitext(pred_file)[0]
    
    # Lettura dei ground truth
    gt_results = read_ground_truth(os.path.join(ground_truth_folder, gt_file))
    
    # Lettura delle predizioni
    with open(os.path.join(predictions_folder, pred_file), 'r') as file:
        file_content = file.read()
    
    data = extract_json_from_file(os.path.join(predictions_folder, pred_file))
    prediction_results = filter_greater_than_zero(data)
    
    # Calcolo delle metriche per immagine
    metrics = calculate_metrics(gt_results, prediction_results, mapping)
    final_metrics_per_image = calculate_classification_metrics(metrics)
    all_metrics.append(final_metrics_per_image)
    
    # Estrai il tempo di elaborazione e aggiungilo al totale
    #processing_time = extract_time(os.path.join(predictions_folder, pred_file))
    #print(processing_time)
    #total_processing_time += processing_time
    #file_count += 1

#total_processing_time = total_processing_time/file_count
#print(f'Average time elapsed per image: {total_processing_time}')
# Calcola le metriche finali
final = calculate_final_metrics(all_metrics)
print(f'final metric: {final}')
save_metrics_to_file(final, output_folder_final, "final")

#########################################################################################################
final_ppes_metrics = {
    'People': {
        'Accuracy': 0, 
        'Precision': 0, 
        'Recall': 0, 
        'F1-Score': 0
    },
    'no-Hard_hats': {
        'Accuracy': 0, 
        'Precision': 0, 
        'Recall': 0, 
        'F1-Score': 0
    },
    'no-Gloves': {
        'Accuracy': 0, 
        'Precision': 0, 
        'Recall': 0, 
        'F1-Score': 0
    },
    'no-Reflective_vests': {
        'Accuracy': 0, 
        'Precision': 0, 
        'Recall': 0, 
        'F1-Score': 0
    },
    'no-Security_boots': {
        'Accuracy': 0, 
        'Precision': 0, 
        'Recall': 0, 
        'F1-Score': 0
    },
}
# Second part of the code to calculate per DPI metrics
gt_files = sorted(os.listdir(ground_truth_folder))
pred_files = sorted(os.listdir(predictions_folder)) 

# Iterate over each DPI type
for dpi in DPI:
    all_metrics_per_dpi = []  # Initialize a list to collect metrics for each DPI
    for gt_file, pred_file in zip(gt_files, pred_files):
        base_filename = os.path.splitext(pred_file)[0]

        # Read ground truth and predictions for the current image
        gt_results = read_ground_truth(os.path.join(ground_truth_folder, gt_file))
        data = extract_json_from_file(os.path.join(predictions_folder, pred_file))
        
        # Filter predictions with counts greater than zero
        prediction_results = filter_greater_than_zero(data)
        
        # Calculate per-image metrics for the current DPI
        metrics = calculate_metrics_per_DPI(gt_results, prediction_results, mapping, dpi)
        final_metrics_per_image = calculate_classification_metrics(metrics)
        all_metrics_per_dpi.append(final_metrics_per_image)

    # Calculate final averaged metrics for the current DPI
    final_dpi_metrics = calculate_final_metrics(all_metrics_per_dpi)
    print(f'final metric {dpi}: {final_dpi_metrics}')
    
    # Update the final PPE metrics dictionary with calculated values
    final_ppes_metrics[dpi]['Accuracy'] = final_dpi_metrics['final_accuracy']
    final_ppes_metrics[dpi]['Precision'] = final_dpi_metrics['final_precision']
    final_ppes_metrics[dpi]['Recall'] = final_dpi_metrics['final_recall']
    final_ppes_metrics[dpi]['F1-Score'] = final_dpi_metrics['final_f1_score']

# Save final PPEs metrics to file
output_file_path = os.path.join(output_folder_final, "final_ppes_metrics.json")
with open(output_file_path, 'w') as f:
    json.dump(final_ppes_metrics, f, indent=4)

final metric: {'final_accuracy': 0.530010704031445, 'final_precision': 0.7472736606715937, 'final_recall': 0.7040053856214122, 'final_f1_score': 0.6752085386253803}

final metric People: {'final_accuracy': 0.9367896578529697, 'final_precision': 0.9770588538445681, 'final_recall': 0.9564840507616482, 'final_f1_score': 0.959570853981972}

final metric no-Hard_hats: {'final_accuracy': 0.26617836528550815, 'final_precision': 0.33617037724180576, 'final_recall': 0.280657338693053, 'final_f1_score': 0.2871924435336642}

final metric no-Gloves: {'final_accuracy': 0.3710196954920982, 'final_precision': 0.6498518346732632, 'final_recall': 0.3964925361435102, 'final_f1_score': 0.44568886850309025}

final metric no-Reflective_vests: {'final_accuracy': 0.46871255097716136, 'final_precision': 0.6757000394987409, 'final_recall': 0.4780774465433556, 'final_f1_score': 0.5156325867297747}

final metric no-Security_boots: {'final_accuracy': 0.1353208702425663, 'final_precision': 0.2777056277056277, 'fin